# Exporting high quality satellite images <img align="right" src="../Supplementary_data/dea_logo.jpg">

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with both the `DEA Sandbox` and `NCI` environments
* **Products used:** 
[ga_ls5t_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3),
[ga_ls7e_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls7e_ard_3),
[ga_ls8c_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3),
[ga_ls9c_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls9c_ard_3),
[ga_s2am_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_s2am_ard_3),
[ga_s2bm_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_s2bm_ard_3)


## Background
Most of the case studies in this repository focus on quantitatively analysing satellite data to obtain insights into Australia's changing environment. 
However, satellite imagery also represents a powerful tool for visualisation.
Images taken by satellites can help explain physical processes, highlight change over time, or provide valuable context to better understand the impacts of recent environmental events such as flooding or fire. 
Satellite data can also be processed to create images of the landscape based on invisible wavelengths of light (e.g. false colour images), allowing us to obtain richer insights into features and processes that would otherwise be invisible to the human eye.

### Digital Earth Australia use case

**Digital Earth Australia** provides over three decades of satellite imagery across the entire continent of Australia. Satellite data from the [NASA/USGS Landsat program](https://www.usgs.gov/land-resources/nli/landsat) allow us to produce fortnightly images of Australia's diverse natural and artificial landscapes at any time since 1986. More recently, the [Copernicus Sentinel-2 mission](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) has provided even higher resolution imagery as frequently as every 5 days since 2015. 





## Description
This notebook provides an interactive tool for selecting, loading, processing and exporting satellite imagery as a high quality image file. 
This can be used in combination with the interactive [Digital Earth Australia Maps](maps.dea.ga.gov.au) platform to identify an image of interest, then download it using this notebook for use in other applications.

The tool requires no coding experience. 
It supports Sentinel-2 and Landsat data, creating True and False colour images, and advanced features like applying pansharpening to increase the resolution of Landsat 7, 8 and 9 imagery, power transformations to darken bright features, or unsharp masking to sharpen imagery.


***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages used for the analysis.

In [1]:
import sys
sys.path.insert(1, '../Tools/')
from dea_tools.app import imageexport

## Digital Earth Australia satellite image export app

Run the `imageexport.imageexport_app()` cell below to launch the interactive application that is used to select the area to load and export satellite imagery.

### Selecting parameters 

On the left of the map, you will see a date selector, and a list of satellite imagery you can load:

* **Landsat**: Data from the Landsat 5, 7, 8 and 9 satellites
* **Sentinel-2**: Data from the Sentinel-2A and 2B satellites

Satellite imagery can be viewed and exported in two colour styles:

* **True colour**: Creates a true colour image using the red, green and blue satellite bands
* **False colour**: Creates a false colour image using short-wave infrared, infrared and green satellite bands

You can also modify the brightness and contrast of the output image by dragging the `Colour stretch` slider, and select the output image format from `.JPG` (smaller file sizes, lower quality) to `.PNG` (larger file sizes, higher quality).

### Selecting a location to export
Zoom into the map until you see imagery appear (imagery may appear as diagonal grey stripes until you zoom in close enough for it to load). 
Select the `Draw a rectangle` tool on the left of the map, and draw a shape around the area you are interested in. 
When you are ready, press the green `Export imagery` button on the bottom left to start the image export. 

> To keep load times reasonable, the app is restricted to image exports **smaller than 10000 square kilometers** in size. This limit can be overuled in the `Advanced` settings.

In [2]:
imageexport.imageexport_app()

imageexport_app(children=(GridspecLayout(children=(HTML(value='<h3>Digital Earth Australia satellite image exp…

## Advanced settings

The image exporter application supports several advanced features. 
To access these, click on the `Advanced` tab in the menu to the left of the map to expand it:
* `Resolution`: The spatial resolution to load data (in metres). By default, the tool will automatically set the best possible resolution depending on the satellites selected (i.e. 30 m for Landsat, 10 m for Sentinel-2). 
Increasing this (e.g. to `100`) can be useful for loading large spatial extents.
* `Pansharpen Landsat`: Whether to apply pansharpening (using the Brovey Transform) to increase the resolution of Landsat imagery from 30 m to 15 m pixels. This is only possible when exporting data from Landat 7, Landsat 8 or Landsat 9 (not Landsat 5 or Sentinel-2).
* `Apply power transformation`: Raises imagery by a power to reduce bright features and enhance dark features. 
This can add extra definition over areas with extremely bright features like snow, beaches or salt pans.
* `Apply unsharp masking` Whether to apply unsharp masking to increase the sharpness of the output image. 
If activated, you can specify both the radius (in pixels) and the amount/strength of the sharpening.
* `Override maximum size limit`: Whether to override the app's default 10,000 square km area limit.
This can be used to load larger areas of satellite imagery, but should be used with caution as it can lead to memory issues or crashes.

> **If running the notebook for the first time**, keep the default settings below. This will demonstrate how the analysis works and provide meaningful results. 

## Downloading exported image

The image export will be completed when `Finished exporting image` appears above, and a preview of your image is shown below the map.

The high resolution image file generated above will be saved to the same location you are running this notebook from (e.g. typically `Real_world_examples`). 
In JupyterLab, use the file browser to locate the image file with a name in the following format: 

`Landsat-8 - 2021-03-31 - Canberra, Australian Capital Territory - True colour, 30 m resolution.png`

If you are using the **DEA Sandbox**, you can download the image to your PC by right clicking on the image file and selecting `Download`.

## Next steps
When you have exported your first image, modify some parameters and rerun the analysis by clicking `Export imagery`. 
For example, you could try:

* Select imagery from a new date using the date selector. 
* Changing the satellite data to `"Sentinel-2"` to export a Sentinel-2 image instead of Landsat.
* Selecting the `"False colour"` style to export a false colour view of the landscape that highlights growing vegetation and water.
* Specify a custom resolution in the `Advanced` tab, e.g. `1000`.
* Experiment with different colour stretches and the `Advanced` tab unsharp masking and power transformation functionality to alter the appearance of the resulting image.


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** January 2023

**Compatible datacube version:** 

In [3]:
import datacube
print(datacube.__version__)

1.8.6


## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)